In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import wfdb
import ast
import json
import os
import tqdm

In [ ]:
root_directory_path = '/home/lucas/Desktop/programming/classwork/Senior_project/project/data'
sig_database = pd.read_csv(os.path.join(root_directory_path, 'ptbxl_database.csv'), index_col='ecg_id')
sig_database.columns


In [ ]:
sig_database = sig_database.drop(
    ['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site', 'device', 'recording_date', 'validated_by', 'second_opinion', 'initial_autogenerated_report', 'validated_by_human','extra_beats', 'pacemaker'],
    axis=1
)
print(sig_database.columns)

In [ ]:
def load_one_sample(path):
    data = wfdb.rdsamp(path)
    signals = np.asmatrix(data[0])
    meta = data[1]
    return signals, meta

one_sample = sig_database.iloc[0]
file_path = one_sample['filename_lr']
signals, meta = load_one_sample(os.path.join(root_directory_path, file_path))
print(signals, meta)
signal = signals.flatten().tolist()[0]
print(signal)
plt.plot(range(len(signal)), signal)
plt.show()

# Infarction Stadium Labels
Infarction Stadium details how damaged the heart tissue is.
* I used the methodology described in this paper to distill the labels: https://arxiv.org/pdf/2306.15681

In [ ]:
infarction_labels = sig_database['infarction_stadium1']
print(set(infarction_labels))

In [ ]:
def fix_infarction_label(label):
    if pd.isna(label) or label == "unknown":
        return "None"
    if label == "Stadium I-II":
        return "Stadium I"
    if label == "Stadium II-III":
        return "Stadium II"
    return label
    
infarction_labels = infarction_labels.apply(lambda x: fix_infarction_label(x))
print(set(infarction_labels))

In [ ]:
sig_database = sig_database.drop(
    ['infarction_stadium1', 'infarction_stadium2'],
    axis=1  # Drop columns
)
sig_database['infarction_labels'] = infarction_labels
sig_database.columns

In [ ]:
set(sig_database.heart_axis.values.flatten())

In [ ]:
codes = sig_database[['scp_codes']]
codes.head()

In [ ]:
unique_codes = set()
for i in range(len(codes)):
    code_dict = ast.literal_eval(codes.iloc[i]['scp_codes'])
    for key in code_dict.keys():
        unique_codes.add(str(key))
unique_codes = sorted(list(unique_codes))

In [ ]:
print(len(unique_codes))